In [1]:
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
day = datetime.datetime.strftime(datetime.datetime.now(), "%Y_%m_%d")
day

'2022_09_19'

In [3]:

! rm -rf "eqList/eqList"$day".xls"
! cp "/Users/ivan/Downloads/eqList"$day".xls" "eqList/eqList"$day".xls"
! ls -l "eqList/"

aeta = f"eqList/eqList{day}.xls"
aeta

total 87392
-rw-r--r--@ 1 ivan  staff   565464  7 30 18:56 eqList2022_07_30.xlsx
-rw-r--r--@ 1 ivan  staff   566439  8  9 01:05 eqList2022_08_09.xlsx
-rw-r--r--@ 1 ivan  staff  3567272  8 11 11:27 eqList2022_08_11.xls
-rw-r--r--@ 1 ivan  staff   566428  8 11 00:45 eqList2022_08_11.xlsx
-rw-r--r--@ 1 ivan  staff  3568288  8 12 16:56 eqList2022_08_12.xls
-rw-r--r--@ 1 ivan  staff  3569648  8 13 15:09 eqList2022_08_13.xls
-rw-r--r--@ 1 ivan  staff  3573668  8 15 16:06 eqList2022_08_15.xls
-rw-r--r--@ 1 ivan  staff  3575011  8 16 18:58 eqList2022_08_16.xls
-rw-r--r--@ 1 ivan  staff  3576341  8 19 17:36 eqList2022_08_19.xls
-rw-r--r--@ 1 ivan  staff  3581086  8 26 10:12 eqList2022_08_26.xls
-rw-r--r--@ 1 ivan  staff  3585174  9  2 01:05 eqList2022_09_02.xls
-rw-r--r--@ 1 ivan  staff  3590551  9  5 16:37 eqList2022_09_05.xls
-rw-r--r--@ 1 ivan  staff  3603008  9 11 16:27 eqList2022_09_11.xls
-rw-r--r--@ 1 ivan  staff  3608412  9 17 20:52 eqList2022_09_17.xls
-rw-r--r--@ 1 ivan  staff  361502

'eqList/eqList2022_09_19.xls'

In [11]:
N = 3
T = 10
# 经度
lonL, lonH = 98, 107
# 纬度
latL, latH = 22, 34 


In [12]:

import xml
DOMTree = xml.dom.minidom.parse(aeta).documentElement
Row = DOMTree.getElementsByTagName("Row")

cols, data = [], []
for nRow, iRow in enumerate(Row):
    if nRow == 0:
        for jRow in iRow.getElementsByTagName("Cell"):
            cols.append(jRow.childNodes[0].childNodes[0].data)
    else:
        idata = []
        for jRow in iRow.getElementsByTagName("Cell"):
            idata.append(jRow.childNodes[0].childNodes[0].data)
        data.append([float(dj) if nj in [1,2,3,4] else str(dj) for nj ,dj in enumerate(idata)])

data = pd.DataFrame(data, columns=cols)
print(data.dtypes)
data.head(T)


发震时刻       object
震级(M)     float64
纬度(°)     float64
经度(°)     float64
深度(千米)    float64
参考位置       object
dtype: object


,发震时刻,震级(M),纬度(°),经度(°),深度(千米),参考位置
0,2022-09-19 10:07:43,5.7,23.42,121.35,10.0,台湾花莲县
1,2022-09-19 04:21:15,3.0,40.03,78.06,20.0,新疆克孜勒苏州阿图什市
2,2022-09-18 23:06:41,3.6,36.08,77.93,10.0,新疆和田地区皮山县
3,2022-09-18 20:57:56,3.9,28.86,95.89,10.0,西藏林芝市墨脱县
4,2022-09-18 17:41:49,5.5,-10.80,164.20,20.0,圣克鲁斯群岛
5,2022-09-18 17:39:55,5.4,23.28,121.34,10.0,台湾花莲县
6,2022-09-18 16:46:32,4.9,23.14,121.23,15.0,台湾花莲县
7,2022-09-18 14:57:35,4.8,23.09,121.26,10.0,台湾台东县
8,2022-09-18 14:44:14,6.9,23.15,121.30,10.0,台湾花莲县
9,2022-09-18 14:32:01,5.0,23.17,121.20,10.0,台湾台东县


In [13]:
def regeo(location):
    try:
        import json
        import requests

        host = 'https://regeo.market.alicloudapi.com'
        path = '/v3/geocode/regeo'
        appcode = '830fa522a512421eaa202bb80afe8921'
        querys = f'location={location}' # 经度在前，纬度在后
        with requests.get(
            f"{host}{path}?{querys}", 
            headers={'Authorization': f'APPCODE {appcode}'}
        ) as r:
            j = json.loads(r.content.decode())["regeocode"]["formatted_address"]
            return j
    except:
        return "NaN"

In [14]:

Kdata = []
for STATION in tqdm(data.head(T)["参考位置"]):
    # idata
    idata = data[data["参考位置"] == STATION][["参考位置", "发震时刻"]]
    idata.rename({
        "参考位置": "初次参考位置", 
        "发震时刻": "初次发震时刻",
    }, axis=1, inplace=True)
    idata["K"] = 1
    # print(idata.shape)

    # jdata
    jdata = data[
        (data["参考位置"] != STATION) & 
        (data["经度(°)"] >= lonL) & (data["经度(°)"] <= lonH) &
        (data["纬度(°)"] >= latL) & (data["纬度(°)"] <= latH) &
        # (data["震级(M)"] >= 3.5) &
        True
    ][["参考位置", "震级(M)", "纬度(°)", "经度(°)", "发震时刻"]]
    jdata["K"] = 1
    # print(jdata.shape)

    # k0data
    k0data = pd.merge(
        idata,
        jdata,
        on="K"
    )
    # print(k0data.shape)

    # k1data
    # 筛选近N天内
    k0data["gap-发震时刻"] = (
        pd.to_datetime(k0data["发震时刻"]) - 
        pd.to_datetime(k0data["初次发震时刻"])
    ).dt.days

    k1data = (k0data["gap-发震时刻"] >= 1) & (k0data["gap-发震时刻"] <= N)
    # print(pd.value_counts(k1data))

    k1data = k0data[k1data]
    # print(k1data.shape)

    # k2data
    k2data = k1data.groupby(
        ["初次参考位置", "初次发震时刻", "K"], 
        as_index=False
    ).agg({
        "震级(M)": "mean", # 特殊/余震
        "纬度(°)": "mean",
        "经度(°)": "mean",
    }).groupby(
        ["K"], 
        as_index=False
    ).agg(
        {
        "K": "sum",
        "震级(M)": "mean", 
        "纬度(°)": "mean",
        "经度(°)": "mean",
    })

    k2data["参考位置"] = STATION
    k2data["K%"] = k2data["K"]/idata.shape[0]
    k2data["震级C"] = k2data["震级(M)"].apply(lambda x: "" if x >= 3.5 else "❌")
    k2data["经度C"] = k2data["经度(°)"].apply(lambda x: "" if x >= lonL and x <= lonH else "❌")
    k2data["纬度C"] = k2data["纬度(°)"].apply(lambda x: "" if x >= latL and x <= latH else "❌")
    k2data["经纬度解析"] = [regeo(f"{_1},{_2}") for _1, _2 in zip(k2data["经度(°)"], k2data["纬度(°)"])]
    k2data["经度(°)"] = k2data["经度(°)"].apply(lambda x: f"{x:.6f}")
    k2data["纬度(°)"] = k2data["纬度(°)"].apply(lambda x: f"{x:.6f}")

    k2data = k2data[
        ['参考位置', '震级(M)', '震级C', '纬度(°)', '纬度C', '经度(°)', '经度C', 'K', 'K%','经纬度解析']
    ]
    # print(k2data)
    Kdata.append(k2data)

    # 
    del idata, jdata, k0data, k1data, k2data


100%|██████████| 10/10 [00:03<00:00,  3.06it/s]


In [15]:
endata = pd.merge(
    data.head(T).rename(
        {icol: f"#{icol}#" for icol in data.columns if icol != "参考位置"}, 
        axis=1
    ), 
    pd.concat(Kdata),
    on="参考位置"
).sort_values("#发震时刻#", ascending=False)

endata

,#发震时刻#,#震级(M)#,#纬度(°)#,#经度(°)#,#深度(千米)#,参考位置,震级(M),震级C,纬度(°),纬度C,经度(°),经度C,K,K%,经纬度解析
0,2022-09-19 10:07:43,5.7,23.42,121.35,10.0,台湾花莲县,3.481905,❌,28.028614,,102.229708,,41,0.585714,四川省凉山彝族自治州喜德县李子乡蒋家沟
1,2022-09-19 10:07:43,5.7,23.42,121.35,10.0,台湾花莲县,3.481905,❌,28.028614,,102.229708,,41,0.585714,四川省凉山彝族自治州喜德县李子乡蒋家沟
2,2022-09-19 10:07:43,5.7,23.42,121.35,10.0,台湾花莲县,3.481905,❌,28.028614,,102.229708,,41,0.585714,四川省凉山彝族自治州喜德县李子乡蒋家沟
3,2022-09-19 10:07:43,5.7,23.42,121.35,10.0,台湾花莲县,3.481905,❌,28.028614,,102.229708,,41,0.585714,四川省凉山彝族自治州喜德县李子乡蒋家沟
16,2022-09-19 04:21:15,3.0,40.03,78.06,20.0,新疆克孜勒苏州阿图什市,3.331333,❌,28.740569,,102.731095,,65,0.714286,四川省凉山彝族自治州甘洛县斯觉镇
17,2022-09-18 23:06:41,3.6,36.08,77.93,10.0,新疆和田地区皮山县,3.327854,❌,28.723398,,102.904413,,59,0.614583,四川省凉山彝族自治州甘洛县吉米镇约洛布哈
18,2022-09-18 20:57:56,3.9,28.86,95.89,10.0,西藏林芝市墨脱县,3.554113,,29.099329,,104.226905,,11,0.687500,四川省宜宾市叙州区泥溪镇板栗沟
19,2022-09-18 17:41:49,5.5,-10.80,164.20,20.0,圣克鲁斯群岛,3.556398,,28.901725,,103.519243,,27,0.750000,四川省乐山市马边彝族自治县劳动镇桂花
6,2022-09-18 17:39:55,5.4,23.28,121.34,10.0,台湾花莲县,3.481905,❌,28.028614,,102.229708,,41,0.585714,四川省凉山彝族自治州喜德县李子乡蒋家沟
7,2022-09-18 17:39:55,5.4,23.28,121.34,10.0,台湾花莲县,3.481905,❌,28.028614,,102.229708,,41,0.585714,四川省凉山彝族自治州喜德县李子乡蒋家沟


In [16]:
# 
day1 = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=1), "%Y-%m-%d")
dayn = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=N), "%Y-%m-%d")
day7 = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=7), "%Y-%m-%d")
print(f">>> {day1} To {day7} / {dayn}")

message = f"""
# 有震预测 earthquake prediction
check_my_prediction(myToken, '{day1}', '{dayn}', 1, latitude=$_1, longitude=$_2, magnitude=$_3)
"""
endata = endata[endata["震级C"] != "❌"]
for _1, _2, _3 in zip(endata["纬度(°)"], endata["经度(°)"], endata["震级(M)"]):
    print(message
          .replace("$_1",f"{float(_1):.6f}")
          .replace("$_2",f"{float(_2):.6f}")
          .replace("$_3",f"{float(_3):.1f}")
         )

message = f"""
# 无震预测 No earthquake prediction
check_my_prediction(myToken, '{day1}', '{day7}', 0)
"""
print(message)
print()


>>> 2022-09-20 To 2022-09-26 / 2022-09-22

# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-09-20', '2022-09-22', 1, latitude=29.099329, longitude=104.226905, magnitude=3.6)


# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-09-20', '2022-09-22', 1, latitude=28.901725, longitude=103.519243, magnitude=3.6)


# 无震预测 No earthquake prediction
check_my_prediction(myToken, '2022-09-20', '2022-09-26', 0)




In [17]:
! cp Test002.ipynb "back/Test002-"$day".ipynb"
! ls -l back/

total 512
-rw-r--r--@ 1 ivan  staff  34490  8 16 18:58 Test002-2022_08_16.ipynb
-rw-r--r--@ 1 ivan  staff  33275  8 19 17:36 Test002-2022_08_19.ipynb
-rw-r--r--@ 1 ivan  staff  33096  8 26 10:12 Test002-2022_08_26.ipynb
-rw-r--r--@ 1 ivan  staff  29782  9  2 01:05 Test002-2022_09_02.ipynb
-rw-r--r--@ 1 ivan  staff   8874  9  5 16:37 Test002-2022_09_05.ipynb
-rw-r--r--@ 1 ivan  staff  43740  9 11 16:27 Test002-2022_09_11.ipynb
-rw-r--r--@ 1 ivan  staff  32193  9 17 20:52 Test002-2022_09_17.ipynb
-rw-r--r--@ 1 ivan  staff  27561  9 19 17:47 Test002-2022_09_19.ipynb
